# Portfolio Hedging with PCA / factors
We build synthetic multi-asset returns, extract principal components, and hedge exposures.

In [ ]:
import numpy as np, matplotlib.pyplot as plt
np.random.seed(42)
T=2000; N=5
f=np.cumsum(np.random.randn(T)*0.5)
X=np.vstack([f + 0.2*np.cumsum(np.random.randn(T)) + 0.1*np.random.randn(T) for _ in range(N)]).T


## 1. Covariance & eigen decomposition

In [ ]:
C=np.cov(X, rowvar=False)
val,vec=np.linalg.eigh(C)
idx=np.argsort(val)[::-1]
val,vec=val[idx],vec[:,idx]


## 2. First factor weights

In [ ]:
w=vec[:,0]; w=w/np.linalg.norm(w)
print('weights',w)


## 3. Hedge one asset using factor

In [ ]:
y=X[:,0]; f1=X@w
beta=np.cov(y,f1)[0,1]/np.var(f1)
res=y-beta*f1
plt.figure(); plt.plot(res); plt.title('Hedged series'); plt.show()


## 4. Rolling beta & residual volatility

In [ ]:
win=200
betas=[]; vol=[]
for i in range(len(y)-win):
    yy=y[i:i+win]; ff=f1[i:i+win]
    b=np.cov(yy,ff)[0,1]/np.var(ff)
    betas.append(b); vol.append(np.std(yy-b*ff))
plt.figure(); plt.plot(betas); plt.title('Rolling beta');
plt.figure(); plt.plot(vol); plt.title('Rolling residual vol'); plt.show()


## 5-10. Backtest signals on residuals

In [ ]:
res=z=np.copy(res)
mu=np.convolve(res,np.ones(200)/200,'same')
sig=np.sqrt(np.convolve((res-mu)**2,np.ones(200)/200,'same'))
z=(res-mu)/(sig+1e-9)
pos=(z<-2).astype(int)-(z>2).astype(int)
pnl=np.cumsum(np.diff(res,prepend=res[0])*pos)
plt.figure(); plt.plot(z); plt.title('Z of residual');
plt.figure(); plt.plot(pnl); plt.title('PnL residual strategy'); plt.show()
